# 네이버 영화평점

## 개요
- https://movie.naver.com/ 로 이동
- 평점/리뷰 메뉴 클릭

## 요청방식
- url
    - https://movie.naver.com/movie/point/af/list.nhn?&page=2
- 요청방식
    - get
- 요청파라미터
    - page: 페이지 번호
    - 1000 페이지 까지만 조회가능
    
## 항목
- base : table.list_netizen > tbody > tr > td.title  : 이 td들 안에 다 있다.
- 영화제목 : > `a.movie : text`
    - href속성은 그 영화의 댓글만 모아놓은 url( 상대경로로 href="?st=mcode&sword=185917&target=after" 형식으로 쿼리 스트링만 있다.)
- 평점
    - > `div.list_netizen_score > em` 의 text
- 댓글
    - br의 next_sibling
    - text조회하면 안의 하위 태그들의 text도 조회된다.
        - **br을 찾은 뒤 그 다음 노드로 찾는다.**
            - tag.next_sibling : 텍스트 노드까지 포함 해서 다음 노드를 반환한다.
                - br다음의 text node를 조회할 것이므로 next_sibling 사용
            - tag.next_element : 다음 element node 반환
            
```html
<td class="title">
			
			
				<a href="?st=mcode&amp;sword=189633&amp;target=after" class="movie color_b">야구소녀</a>
			
			
			
			<div class="list_netizen_score">
				<span class="st_off"><span class="st_on" style="width:100%">별점 - 총 10점 중</span></span><em>10</em>
			</div>
			<br>기대없이 시청하는데 나도모르게 눈물을 흘렸네요 
			
			
			
				
				
				
				<a href="javascript:report('mkbk****', '3wfKF9kn6UVmVCmf5aXfMSpd8o/GJ1e3/VWl/7hODlk=', '기대없이 시청하는데 나도모르게 눈물을 흘렸네요', '17024530', 'point_after');" class="report" style="color:#8F8F8F" title="새 창">신고</a>
			
			
			</td>
```



### 1페이지 조회

In [14]:
import requests
from urllib import parse
from bs4 import BeautifulSoup
base_url = 'https://movie.naver.com/movie/point/af/list.nhn?&page={}'
url = base_url.format(1)
res=requests.get(url)

if res.status_code == 200:
    soup = BeautifulSoup(res.text)
    tds = soup.select('table.list_netizen > tbody > tr > td.title')
    print(len(tds))
    for td in tds:
        movie_title = td.select_one('a.movie').text.strip()
        link = td.select_one('a.movie').get('href')
        link = parse.urljoin(base_url, link)
        score = td.select_one('div.list_netizen_score > em').text.strip()
        comment = td.select_one('br').next_sibling.strip() #textnode 인 경우 text 로 조회할 필요 없다.
        print(movie_title, link, score, comment, sep=' :: ')
        print('-------------------------------------------------')

10
성실한 나라의 앨리스 :: https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=118323&target=after :: 8 :: 잘봤습니다~이정현연기~~~짱
-------------------------------------------------
강철비2: 정상회담 :: https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=188909&target=after :: 10 :: 일베가 좋으면 공감, 싫으면 비공감 누르세요.
-------------------------------------------------
강철비2: 정상회담 :: https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=188909&target=after :: 10 :: 솔직히 큰기대 안했는데 꽤 재밌게봄잠수함씬 와우 잘봤고 전체적으로 이야기 전개도 나쁘지않음
-------------------------------------------------
미스터 고 :: https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=75397&target=after :: 10 :: 
-------------------------------------------------
더블 타겟 :: https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=65113&target=after :: 10 :: 엄청 재밌네요. 긴장감, 스릴감, 제대로 복수..
-------------------------------------------------
작전 :: https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=50176&target=after :: 10 :: 한국 주식영화의 시

## 전체 페이지 조회

In [23]:
import random
random.uniform(0.2, 1.2) #이 사이의 실수를 동일한 확률로 나오게 한다.

0.2934325558951632

In [8]:
import requests
import time
import random
from bs4 import BeautifulSoup


base_url = 'https://movie.naver.com/movie/point/af/list.nhn?&page={}'
#결과 저장할 리스트
comment_list = []
for page in range(1, 101):
    url = base_url.format(page)
    res=requests.get(url)
    if res.status_code == 200:
        soup = BeautifulSoup(res.text, 'lxml')
        tds = soup.select('table.list_netizen > tbody > tr > td.title')
        for td in tds:
            movie_title = td.select_one('a.movie').text.strip()
            link = td.select_one('a.movie').get('href')
            link = parse.urljoin(base_url, link)
            score = td.select_one('div.list_netizen_score > em').text.strip()
            comment = td.select_one('br').next_sibling.strip()
            # 리스트에 저장
            comment_list.append((movie_title, link, score, comment))
        interval = round(random.uniform(0.2, 1.2), 2)
        time.sleep(interval)
print('종료')        

종료


In [9]:
len(comment_list)

1000

In [11]:
import pandas as pd
df = pd.DataFrame(comment_list, 
                  columns=['영화제목','영화링크', '평점','댓글'])
df.to_csv('naver_comment.csv', encoding='utf-8', index=False)

In [15]:
df.to_pickle('naver_comment.pkl',)

In [12]:
df.head()

,영화제목,영화링크,평점,댓글
0,반도,https://movie.naver.com/movie/point/af/list.nh...,2,"오글거리는 대사, 억지 신파, 캐릭터의 역할 모두 엉망."
1,강철비2: 정상회담,https://movie.naver.com/movie/point/af/list.nh...,10,연기가 아주 좋았어요 ㅎㅎ
2,엑시트,https://movie.naver.com/movie/point/af/list.nh...,10,킬링타임용 ㅋ 완전 재밌음
3,반도,https://movie.naver.com/movie/point/af/list.nh...,10,
4,레토,https://movie.naver.com/movie/point/af/list.nh...,10,
